In [1]:
from os.path import join as pjoin
import numpy as np

In [2]:
dirname = '/data101/bartlett/ili/cmass/quijote_z0/pinocchio/L1000-N128/3/'
filename = pjoin(dirname, 'pinocchio.0.5000.pinocchio-L1000-N128-3.snapshot.out')

data = {}

with open(filename, 'rb') as f:

    # Function to read a block
    def read_block(expected_name, dtype, count):
        initial_block_size = np.fromfile(f, dtype=np.int32, count=1)[0]
        block_name = np.fromfile(f, dtype='S4', count=1)[0].decode().strip()
        block_size_with_name = np.fromfile(f, dtype=np.int32, count=1)[0]
        if block_name != expected_name:
            raise ValueError(f"Expected block name '{expected_name}', but got '{block_name}'")
        data_block = np.fromfile(f, dtype=dtype, count=count)
        trailing_block_size = np.fromfile(f, dtype=np.int32, count=1)[0]
        return data_block

    # Read the HEADER block
    header_dtype = np.dtype([
        ('dummy', np.int64),
        ('NPart', np.uint32, 6),
        ('Mass', np.float64, 6),
        ('Time', np.float64),
        ('RedShift', np.float64),
        ('flag_sfr', np.int32),
        ('flag_feedback', np.int32),
        ('NPartTotal', np.uint32, 6),
        ('flag_cooling', np.int32),
        ('num_files', np.int32),
        ('BoxSize', np.float64),
        ('Omega0', np.float64),
        ('OmegaLambda', np.float64),
        ('HubbleParam', np.float64),
        ('flag_stellarage', np.int32),
        ('flag_metals', np.int32),
        ('npartTotalHighWord', np.uint32, 6),
        ('flag_entropy_instead_u', np.int32),
        ('flag_metalcooling', np.int32),
        ('flag_stellarevolution', np.int32),
        ('fill', np.int8, 52)
    ])
    header_block = read_block('HEAD', header_dtype, 1)[0]
    data['header'] = {name: header_block[name] for name in header_dtype.names}

    # Number of particles
    num_particles = header_block['NPart'][1]

    # Read INFO block
    info_dtype = np.dtype([('name', 'S4'), ('type', 'S8'), ('ndim', np.int32), ('active', np.int32, 6)])
    d = read_block('INFO', info_dtype, 4)
    
    # Read empty FMAX block
    fmax_dtype = np.dtype([('dummy', np.int64), ('fmax', np.float32)])
    d = read_block('FMAX', fmax_dtype, 2)
    
    # Read empty RMAX block
    rmax_dtype = np.dtype([('dummy', np.int64), ('rmax', np.int64)])
    d = read_block('RMAX', rmax_dtype, 2)
    
    # Read ID block
    id_dtype = np.dtype([
        ('dummy', np.int64),
        ('ids', np.uint32, num_particles)
    ])
    ids = read_block('ID', id_dtype, 1)
    data['ids'] = ids['ids'][0]
    
    # Read POS block
    pos_dtype = np.dtype([
        ('dummy', np.int64),
        ('pos', np.float32, num_particles * 3)
    ])
    positions = read_block('POS', pos_dtype, 1)
    data['positions'] = positions['pos'].reshape(-1, 3)
    
    # Read VEL block
    vel_dtype = np.dtype([
        ('dummy', np.int64),
        ('vel', np.float32, num_particles * 3)
    ])
    positions = read_block('VEL', vel_dtype, 1)
    data['velocities'] = positions['vel'].reshape(-1, 3)
    
    print(data['ids'][:10])
    for k, v in data['header'].items():
        print(k, v)
    print(data['positions'].shape)
    print(np.amin(data['positions']), np.amax(data['positions']))  
    print(data['velocities'].shape)
    print(np.amin(data['velocities']), np.amax(data['velocities']))

[ 1  2  3  4  5  6  7  8  9 10]
dummy 1099511627784
NPart [      0 2097152       0       0       0       0]
Mass [   0.         4329.04227538    0.            0.            0.
    0.        ]
Time 0.6666666666666666
RedShift 0.5
flag_sfr 0
flag_feedback 0
NPartTotal [      0 2097152       0       0       0       0]
flag_cooling 0
num_files 1
BoxSize 1000.0
Omega0 0.3271
OmegaLambda 0.6729
HubbleParam 0.6313
flag_stellarage 0
flag_metals 0
npartTotalHighWord [0 0 0 0 0 0]
flag_entropy_instead_u 0
flag_metalcooling 0
flag_stellarevolution 0
fill [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
(2097152, 3)
0.000103653816 999.99945
(2097152, 3)
-11863.557 12139.884
